## 서울특별시 구별 인구현황 및 CCTV 현황 분석및 상관관계분석

- pandas 사용법 습득
- 시각화 처리 기법 습득
- 전체적인 분석 방법 습득


+ CCTV 현황 다운 받기 (https://opengov.seoul.go.kr/data/2813905)
+ 서울 열린 데이터 센터에서 인구정보 다운로드 받기 (https://data.seoul.go.kr/dataService/boardList.do)

In [112]:
import pandas as pd

### 1. CCTV현황, 인구현황 데이터 가져오기 

- 필요없는 정보는 빼고 로드한다

In [113]:
s_cctv = pd.read_excel('./data/CCTVstatus.xlsx', encoding='utf-8')
s_pop = pd.read_excel('./data/population.xls', encoding='utf-8') # 15' ~ 18'

In [114]:
s_cctv.tail(2)

,기관명,소계,2013년 이전,2014년,2015년,2016년,2017년
23,중 구,1260,786,40,191,123,120
24,중 랑 구,1053,751,64,102,75,61


In [115]:
s_pop.head(3)

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,세대당인구,65세이상고령자
0,기간,자치구,세대,합계,합계,합계,한국인,한국인,한국인,등록외국인,등록외국인,등록외국인,세대당인구,65세이상고령자
1,기간,자치구,세대,계,남자,여자,계,남자,여자,계,남자,여자,세대당인구,65세이상고령자
2,2015,합계,4189948,10297138,5063550,5233588,10022181,4930943,5091238,274957,132607,142350,2.39,1267563


In [116]:
# 현재 기간, 성별, 세대와 관련된 정보는 필요없으니 필요한 것만 다시 가져오기
s_pop = pd.read_excel('./data/population.xls'
                      , header=2     # 0,1,2 행 중에 2을 header로 사용하자
                      , usecols = 'B, D, G, J, N'
                      , encodinf='utf-8')
s_pop.head(3)

,자치구,계,계.1,계.2,65세이상고령자
0,합계,10297138,10022181,274957,1267563
1,종로구,163822,154986,8836,24892
2,중구,134329,125733,8596,20302


### 2. 컬럼 정보 확인 

- 컬럼명 확인 / 컬럼값 하나 확인하기
- 인덱스 값 확인하기 (행이 몇개 있는지)

In [117]:
cols = s_cctv.columns
cols, type(cols)

(Index(['기관명', '소계', '2013년 이전', '2014년', '2015년', '2016년', '2017년'], dtype='object'),
 pandas.core.indexes.base.Index)

In [118]:
cols[0]

'기관명'

In [119]:
s_cctv.index

RangeIndex(start=0, stop=25, step=1)

### 3. 컬럼 정보 수정하기

In [120]:
# 기관명 컬럼을 '자치구' 컬럼으로 변경하기
cols = s_cctv.columns
s_cctv.rename(columns={cols[0]:'자치구', cols[2]:'2013년이전'}, inplace=True)
s_cctv.head(2)

,자치구,소계,2013년이전,2014년,2015년,2016년,2017년
0,강 남 구,4758,1979,474,760,770,775
1,강 동 구,1493,1028,73,142,240,10


In [121]:
pCols = s_pop.columns
pCols

Index(['자치구', '계', '계.1', '계.2', '65세이상고령자'], dtype='object')

In [122]:
'''
'계' : '인구수',
'계.1' : '한국인' 
'계.2' : '외국인',
'65세이상고령자' : '고령자'
'''

dic = { 
    pCols[1] : '인구수',
    pCols[2] : '한국인',
    pCols[3] : '외국인',
    pCols[4] : '고령자'
}
s_pop.rename(columns=dic, inplace=True)
s_pop.head(2)

,자치구,인구수,한국인,외국인,고령자
0,합계,10297138,10022181,274957,1267563
1,종로구,163822,154986,8836,24892


In [123]:
# 필요없는 데이터 제거하기
s_pop.drop([0], inplace=True)
s_pop.shape

(103, 5)

In [124]:
# 연도별 자치구별 인구수
s_pop_2015 = s_pop[:25]
s_pop_2016 = s_pop[26:51]
s_pop_2017 = s_pop[52:77]
s_pop_2018 = s_pop[78:103]
s_pop_2015.head(2)

,자치구,인구수,한국인,외국인,고령자
1,종로구,163822,154986,8836,24892
2,중구,134329,125733,8596,20302


### 4. 데이터 특성 파악 및 처리

#### 4-1. cctv 데이터 분석 

In [125]:
s_cctv.head(2)

,자치구,소계,2013년이전,2014년,2015년,2016년,2017년
0,강 남 구,4758,1979,474,760,770,775
1,강 동 구,1493,1028,73,142,240,10


In [126]:
# 소계를 중심으로 정렬
s_cctv.sort_values(by=s_cctv.columns[1], ascending=True).head()

,자치구,소계,2013년이전,2014년,2015년,2016년,2017년
9,도 봉 구,899,480,185,49,102,83
2,강 북 구,946,472,70,147,257,0
24,중 랑 구,1053,751,64,102,75,61
3,강 서 구,1202,634,52,177,168,171
17,송 파 구,1203,865,24,124,88,102


+ cctv 증가율을 새롭게 추가해서 자치구별로 cctv가 얼마나 추가되었는지 확인하기
> 증가분 = (2014+2015+2016)/2013 * 100 

#### 4-2. cctv 데이터 처리

+ 누적합 구하기 (인덱싱 vs 슬라이싱)
+ 최근증가율 계산하고 값 추가하기

In [ ]:
# 누적합
